# 

# process for tabular data

In [16]:
input_file = "./../../test_input/WisTMP 7863 (4430-21-00) (1)(1).md"
output_file = "./../../test_output/WisTMP 7863 (4430-21-00) (1)(1).md"

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

cleaned_lines = []

# for i in range (len(lines)):
#     print(i, lines[i])

i = 0
while i < len(lines) - 3:
    if (lines[i].startswith("|") and lines[i+1].strip() == "" and lines[i+2].startswith("|") and lines[i+3].startswith("|-")):
        print("Found misbehaved table header at line", i)

        cleaned_lines.append(lines[i])
        i += 1

        # skip the empty line
        i += 1

        cleaned_lines.append(lines[i])
        i += 1

        # Skip the misplaced separator 
        # i+=1 after this if statement
    else:
        cleaned_lines.append(lines[i])
    i += 1

# Append remaining lines if we stopped early
while i < len(lines):
    cleaned_lines.append(lines[i])
    i += 1



# save the new file 
with open(output_file, 'w', encoding='utf-8') as f:
    f.writelines(cleaned_lines)

Found misbehaved table header at line 96


# Extract comment

In [ ]:
input_file = "./../../test_input/WisTMP 7554 COMMENT.md"
output_file = "./../../test_output/WisTMP 7554 COMMENT_extracted.md"

import re

def extract_section_comments(text):
    """
    Extracts all section comments from the given document text.
    
    Assumes that each section comment block starts with a markdown header
    that contains "**Section Comments**" (e.g., "## **Section Comments**")
    and continues until the next markdown header or the end of the text.
    
    Returns:
        A list of strings, each representing a section comment block.
    """
    # Pattern explanation:
    #   r                 -> raw string literal. Tells Python not to treat backslashes (\) as escape characters.Without the r, you'd have to write double backslashes like "\\s*\\(By [^)]+\\)".
    #   \s                -> whitespace. * -> "zero or more" of the preceding element. \s* -> "zero or more whitespace characters".
    #   ^#{2,6}\s*        -> Matches a markdown header of level 2 to 6 at the start of a line.
    #   \*\*Section Comments\*\* -> Matches the bolded text "Section Comments".
    #   \s*\n             -> Matches the rest of the header line.
    #   (.*?)             -> Lazily captures everything following (the comment block).
    #   (?=^\s*#|\Z)      -> Stops capturing when a new header line is encountered or end of text.
    pattern = r"^#{2,6}\s*\*\*Section Comments\*\*\s*\n(.*?)(?=^\s*#|\Z)"
    
    # Using re.MULTILINE to ensure ^ and $ match start/end of each line,
    # and re.DOTALL so that '.' matches newline characters.
    matches = re.findall(pattern, text, flags=re.MULTILINE | re.DOTALL)
    
    # Clean up each extracted comment block by stripping any extra whitespace.
    return [match.strip() for match in matches]

def remove_date_author(text):
    """
    Removes date and author information from the text.
    
    Looks for any parenthesized statement starting with "By" (e.g., 
    (By Joe Schneider on 02/27/2020 11:04 AM)) and removes it.
    
    Returns:
        The cleaned text without the date and author info.
    """
    # This regex matches patterns like:
    # (By Joe Schneider on 02/27/2020 11:04 AM)
    # It allows for variations in spacing.
    cleaned_text = re.sub(r"\s*\(By [^)]+\)", "", text)
    return cleaned_text


with open(input_file, 'r', encoding='utf-8') as file:
    document_text = file.read()

# Extract all Section Comments blocks
section_comments = extract_section_comments(document_text)

# Display the extracted Section Comments
for idx, comment in enumerate(section_comments, start=1):
    print(f"Section Comments")
    print(comment)
    print("\n")

print("-" * 80)

# Remove date and author information from the text
cleaned_comments = [remove_date_author(comment) for comment in section_comments]

# Display the extracted Section Comments
for idx, comment in enumerate(cleaned_comments, start=1):
    print(f"Section Comments")
    print(comment)
    print("\n")


Section Comments
I sent an updated WZIA to Chris for this project. That needs to be updated prior to 60% approval. (By Joe Schneider on 02 /27/2020 11:04 AM)

Ok, just saw that it was attached below. Please move it to this section. (By Joe Schneider on 02/27/2020 11:05 AM)


Section Comments
As you get further with the estimate on this project please add cost information to this table. You should have the costs for the temp pedestrian accommodations along with the traffic control costs. (By Joe Schneider on 02/20/2020 08:58 AM)

I would think you would have additional cost info to add here (By Joe Schneider on 03/16/2020 02:23 PM)


Section Comments
For flagging operations you can use a capacity of 600 vphpl. For single lane closures in a signalized corridor (assuming multi lane roadway) you would use 900 vphpl. (By Joe Schneider on 02/20/2020 09:01 AM)

Once details for the temp ped accommodations are put together the impact to the roadway will be clearer and we should be able to look